Reinforcement learning with Gymnasium ( Blackjack)

In [ ]:
from __future__ import annotations

from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib.patches import Patch
from tqdm import tqdm

import gymnasium as gym


# Let's start by creating the blackjack environment.
# Note: We are going to follow the rules from Sutton & Barto.
# Other versions of the game can be found below for you to experiment.

env = gym.make("Blackjack-v1", sab=True,render_mode="rgb_array")

In [ ]:
#reset the environment to get the first observation
done = False
observation, info = env.reset()

#observation = (16,9,False)

In [ ]:
# sample a random action from all valid actions
action = env.action_space.sample()
# action=1

# execute the action in our environment and receive infos from the environment
observation, reward, terminated, truncated, info = env.step(action)

# observation=(24, 10, False)
# reward=-1.0
# terminated=True
# truncated=False
# info={}

c:\Users\Fujitsu\anaconda3\lib\site-packages\gymnasium\utils\passive_env_checker.py:249: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [ ]:
q_values = defaultdict(lambda: np.zeros(env.action_space.n))
discount_factor = 0.95
learning_rate = 0.01
n_episodes = 1000
epsilon = 1.0
epsilon_decay = epsilon / (n_episodes / 2)  # reduce the exploration over time
final_epsilon = 0.1

training_error = []
observation, info = env.reset()
# Défini l'action suivante

def get_action(obs):
        """
        Returns the best action with probability (1 - epsilon)
        otherwise a random action with probability epsilon to ensure exploration.
        """
        # with probability epsilon return a random action to explore the environment
        if np.random.random() < epsilon:
            return env.action_space.sample()

        # with probability (1 - epsilon) act greedily (exploit)
        else:
            return int(np.argmax(q_values[obs]))



Algorithm SARSA or SARSA(0) se base sur l'équation ci-après:


    target = reward + (gamma * Q[next_state][next_action])               # construct TD target
    new_value = Q[state][action] + (alpha * (target - current))


In [ ]:
## Mise à jour de la valeur de q par l'algorithm du td learning

def update(
        obs,
        action,
        reward,
        terminated,
        next_obs,
    ):
    """Updates the Q-value of an action."""
    future_q_value = (not terminated) * (q_values[next_obs][action])
    temporal_difference = (
            reward + discount_factor * future_q_value - q_values[obs][action]
        )

    q_values[obs][action] = (
            q_values[obs][action] + learning_rate * temporal_difference
        )
    return q_values[obs][action]


In [ ]:
epsilon = 1.0
n_episodes = 100

final_epsilon = 0.1

## Mise à jour de l'episode
def decay_epsilon(epsilon,episode):
    epsilon_decay = epsilon / (episode / 2)  # reduce the exploration over time
    epsilon = max(final_epsilon,epsilon - epsilon_decay)
    return epsilon


In [ ]:
from IPython.display import clear_output
env = gym.wrappers.RecordEpisodeStatistics(env, deque_size=n_episodes)
for episode in tqdm(range(n_episodes)):
    obs, info = env.reset()
    done = False
    clear_output()

    # play one episode
    while not done:
        # choix de la nouvelle action
        action = get_action(obs)
        next_obs, reward, terminated, truncated, info = env.step(action)

        # update de la q-value
        q_values[obs][action] = update(obs, action, reward, terminated, next_obs)

        ## Visualisation de l'environnement
        frame = env.render()
        plt.imshow(frame)
        plt.show()
        # update if the environment is done and the current obs
        done = terminated or truncated
        obs = next_obs


        decay_epsilon(1.0, 10)

100%|██████████| 100/100 [00:00<00:00, 144.84it/s]
